In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os
import random
import time

import cv2
from torch.utils.data import DataLoader, Dataset
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


data_dir = os.getcwd() + "./trainDataset" # 기존 폴더 경로
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
val_datasets = datasets.ImageFolder(os.path.join(data_dir, 'val'), transforms_test)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=64, shuffle=True, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(val_datasets, batch_size=64, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=64, shuffle=True, num_workers=2)

print('학습 데이터셋 크기:', len(train_datasets))
print('val 데이터셋 크기:', len(val_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

학습 데이터셋 크기: 3582
val 데이터셋 크기: 1534
테스트 데이터셋 크기: 1278
클래스: ['negative', 'positive']


In [3]:
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter
writer = SummaryWriter()

In [4]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.to("cuda") 
#이 오류는 모델의 입력 텐서와 가중치 텐서의 데이터 유형이 일치하지 않을 때 발생합니다. 
#보통 이 오류는 CUDA를 사용하여 GPU에서 모델을 실행할 때 발생합니다.

Using cache found in C:\Users\이승준/.cache\torch\hub\pytorch_vision_v0.10.0
C:\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
#학습
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.95))
num_epochs = 20
model.train()
start_time = time.time()

# 전체 반복(epoch) 수 만큼 반복하며
for epoch in range(num_epochs):
    train_loss = 0.
    train_corrects = 0
    val_loss = 0.
    val_corrects = 0
    # 배치 단위로 학습 데이터 불러오기
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 모델에 입력(forward)하고 결과 계산
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_corrects += torch.sum(preds == labels.data)
    #검증
    for inputs, labels in val_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 모델에 입력(forward)하고 결과 계산
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
        loss.backward()
        optimizer.step()

        val_loss += loss.item() * inputs.size(0)
        val_corrects += torch.sum(preds == labels.data)
    #훈련
    train_loss = train_loss / len(train_datasets)
    train_acc = train_corrects / len(train_datasets) * 100.
    #검증
    val_loss = val_loss / len(val_datasets)
    val_acc = val_corrects / len(val_datasets) * 100.
    writer.add_scalar('Training Loss', train_loss, epoch)
    writer.add_scalar('Val Loss', val_loss, epoch)
    # 학습 과정 중에 결과 출력
    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.2f}s'.format(epoch, train_loss, train_acc, time.time() - start_time))
writer.close()

#0 Loss: 0.6399 Acc: 62.8978% Time: 28.23s
#1 Loss: 0.6253 Acc: 63.1770% Time: 56.14s
#2 Loss: 0.6052 Acc: 66.2758% Time: 83.98s
#3 Loss: 0.5811 Acc: 68.8442% Time: 111.88s
#4 Loss: 0.5652 Acc: 69.2909% Time: 139.80s
#5 Loss: 0.5441 Acc: 71.7755% Time: 167.64s
#6 Loss: 0.5192 Acc: 73.8693% Time: 195.49s
#7 Loss: 0.4917 Acc: 76.0469% Time: 223.41s
#8 Loss: 0.4284 Acc: 79.9553% Time: 251.21s
#9 Loss: 0.3770 Acc: 82.9425% Time: 279.08s
#10 Loss: 0.3325 Acc: 85.6784% Time: 307.07s
#11 Loss: 0.2685 Acc: 89.3914% Time: 334.97s
#12 Loss: 0.2420 Acc: 90.0893% Time: 362.69s
#13 Loss: 0.1969 Acc: 92.0715% Time: 390.51s
#14 Loss: 0.1663 Acc: 93.5511% Time: 418.41s
#15 Loss: 0.1766 Acc: 93.3836% Time: 446.26s
#16 Loss: 0.1357 Acc: 94.8353% Time: 473.98s
#17 Loss: 0.1195 Acc: 95.6170% Time: 501.72s
#18 Loss: 0.1264 Acc: 95.1424% Time: 529.42s
#19 Loss: 0.1012 Acc: 96.2311% Time: 557.23s


In [8]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_dataloader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_dataloader.dataset)

print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_dataloader.dataset), 100. * correct / len(test_dataloader.dataset)))




Test set: Average Loss: -6.1365, Accuracy: 813/1278 (64%)



In [9]:
%load_ext tensorboard
%tensorboard --logdir runs --port=6010